# Process weather indices dataset

This notebook is used to derive a dataset of different indices / extreme events using the bias-corrected CORDEX data provided to Two Bears Environmental Consulting (via KR at SNAP) by Stantec, Inc.

This dataset will consist of four indices (to start), derived at the annual scale for all available models and scenarios on the same grid as the original bias-corrected CORDEX data.

This dataset will be stored in a netCDF file consisting of a dataset for each variable with the following dimensions:

* model
* scenario
* year
* Y
* X

## Indices

The following indices are to be derived, at the annual scale:

* `hd`:  “Hot day” threshold -- the highest observed daily $T_{max}$ such that there are 5 other observations equal to or greater than this value.
* `cd`: “Cold day” threshold -- the lowest observed daily $T_{min}$ such that there are 5 other observations equal to or less than this value.
* `rx1day`: Maximum 1-day precipitation
* `hsd`: Heavy Snow Days –- the mean of the snow totals for the 5 snowiest days
* `su`: Summer Days –- Annual number of days with Tmax above 25 C
* `dw`: Deep Winter days –- Annual number of days with Tmin below -30 C
* `wsdi`: Warm Spell Duration Index -- Annual count of occurrences of at least 5 consecutive days with daily mean T above 90 th percentile of historical values for the date
* `cdsi`: Cold Spell Duration Index -- Same as WDSI, but for daily mean T below 10 th percentile
* `rx5day`: Maximum 5-day precipitation
* `r10mm`: Number of heavy precip days –- Annual count of days with precip > 10 mm
* `cwd`: Consecutive wet days –- Yearly number of the most consecutive days with precip > 1 mm
* `cdd`: Consecutive dry days –- Same as CED, but for days with precip < 1 mm
* `wndd`: Windy Days – Yearly number of days with mean wind speed > 10 m/sec

## Models

The CORDEX data are created by combining a regional climate model with a global circulation model, and there are a couple different types of each represented in this dataset. The combinations are nowhere near exhaustive though, so for our purposes, it should be sufficient to just treat each unique combination as its own "model", of which there are 11:

* CCCma-CanESM2 x CCCma-CanRCM4
* CCCma-CanESM2 x SMHI-RCA4
* CCCma-CanESM2 x UQAM-CRCM5
* ICHEC-EC-EARTH x DMI-HIRHAM5
* ICHEC-EC-EARTH x SMHI-RCA4
* ICHEC-EC-EARTH x SMHI-RCA4-SN
* MPI-M-MPI-ESM-LR x MGO-RRCM
* MPI-M-MPI-ESM-LR x SMHI-RCA4
* MPI-M-MPI-ESM-LR x SMHI-RCA4-SN
* MPI-M-MPI-ESM-MR x UQAM-CRCM5
* NCC-NorESM1-M x SMHI-RCA4

## Processing

Here we now derive this dataset. The strategy will be to iterate over the datasets and read / summarize into summary `xarray.DataArray` objects with matching dimensions, and then combined into one `xarray.Dataset` to then be saved as a netCDF. 

Run the cell below to import the config file which sets paths to directories, makes common imports, etc.

In [1]:
from multiprocessing import Pool
import numpy as np
import tqdm
import xarray as xr
# project
from config import *
import indices
# ignore all-nan slice warnings
import warnings
warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')

Create a list of arguments for filenames and requested summaries:

In [2]:
# this chunk is an artifact of multiprocessing-based optimization attempts
#  but it still serves nicely for utilizing a tqdm progress bar in serial processing
args = []

for scenario in scenarios:
    for varname in varnames:
        for model in models:
            fp = cordex_dir.joinpath(scenario, varname, temp_fn.format(scenario, varname, model))
            
            # aggregate variable names for this particular file
            idx_varnames = idx_varname_lu[varname]
            
            # not all combinations of model, scenario, and model variable actually exist
            if fp.exists():
                args.append((fp, idx_varnames, varname, scenario, model))

We will use functions from the `indices.py` script to derive the indices. Define a wrapper function for the `compute_index` function that will open the connection to a dataset (modeled climate variable) and compute all requested indices for that particular file.

Note - currently, there is only one index computed per variable, but this approach was chosen to facilitate addition of multiple indices per variable.

In [3]:
def run_compute_index(args):
    """Read in data and compute all requested indices for a particular model variable, scenario, and model.
    
    Args:
        fp (path-like): path to the file for the variable required for creating the index variables in indices
        index_list (list): indices to derive using data in provided filepath
        varname (str): model variable being used for indices
        scenario (str): scenario being run
        model (str): model being run
        
    Returns:
        summary_das (tuple): tuple of the form (da, index, scenario, model), where da is a DataArray with dimensions of year (summary year), latitude (lat) and longitude (lon)
    """
    fp, index_list, varname, scenario, model = args
    # passing in model, scenario, agregate variable name
    #  so this information can be handed back after
    #  pool-ing to then construct new Dataset
    
    with xr.open_dataset(fp) as ds:
        out = [indices.compute_index(ds[varname], index, model, scenario) for index in index_list]
    
    return out

Iterate over the arguments created for each index and run. Looks like this seems to be taking ~6 minutes on Atlas using 32 cores if the CORDEX data is available on scratch space:

In [5]:
results = []
    
# trying with less than max number of cpus on Atlas nodes to help with memory allocation errors
with Pool(25) as pool:
    for summary_da in tqdm.tqdm(
        pool.imap_unordered(run_compute_index, args), total=len(args)
    ):
        results.append(summary_da)

 74%|███████████████████████████████████████████████████████████████████████████████████▎                             | 98/133 [06:20<02:16,  3.89s/it]


MemoryError: Unable to allocate 6.72 GiB for an array with shape (153, 68, 365, 95, 5) and data type float32

Merge the `DataArray`s into one `DataSet` (this might take a couple minutes):

In [5]:
ds = xr.merge([da for da_list in results for da in da_list])

Convert data to appropriate units as needed:

In [6]:
# # convert precip indices to mm - conversion factor for precipitation flux to cm (from kg m-2 s-1) provided by John W is 8460.
# ds["rx1day"] = ds["rx1day"] * 86400

# # convert temperature to Celsius
# ds["hd"] = ds["hd"] - 273.15
# ds["cd"] = ds["cd"] - 273.15

Set reasonable data types, converting `nan`s to -9999 as necessary:

In [7]:
def replace_nan(da):
    da.values[np.isnan(da.values)] = -9999
    return da


ds["hsd"] = replace_nan(ds["hsd"]).astype(np.int32)

Round to reasonable precision:

In [8]:
ds["hd"] = np.round(ds["hd"], 1)
ds["cd"] = np.round(ds["cd"], 1)
ds["rx1day"] = np.round(ds["rx1day"], 1)

Add metadata as attributes of the `DataArray`s

In [9]:
ds["hd"].attrs = {
    "units": "C",
    "comment": "'hot day': 6th hottest day of the year",
}
ds["cd"].attrs = {
    "units": "C",
    "comment": "'cold day': 6th coldest day of the year",
}
ds["rx1day"].attrs = {
    "units": "mm",
    "comment": "maximum precipitation for the year",
}
ds["hsd"].attrs = {
    "units": "day",
    "comment": "number of days exceeding 10cm snowfall",
    "_FillValue": -9999,
}

Global metadata:

In [10]:
from datetime import datetime


ds.attrs = {
    "creation_date": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
}

Write to disk (might take a couple of minutes):

In [11]:
ds.to_netcdf(indices_fp)